In [4]:
from glob import glob
import os
import mne
import numpy as np
import pandas
import matplotlib.pyplot as plt
import scipy
from scipy import stats
from features import concatenate_features
from read_data import read_data


In [5]:
%%capture
all_fp=glob('all_SP_edf/*.edf') #the path of all files is in all_fp

healthy_fp= [ i for i in all_fp if 'h' in i.split('\\')[1]] #List of path of all healthy subjects
patient_fp= [ i for i in all_fp if 's' in i.split('\\')[1]]#List path of all sick subjects

healthy_epochs_array=[read_data(i) for i in healthy_fp] # (no of healthy subjects) * (No.of epochs/trials, channels, length of signal)
patient_epochs_array=[read_data(i) for i in patient_fp] # # (no of Sick subjects) * (No.of epochs/trials, channels, length of signal)

data_list=healthy_epochs_array+patient_epochs_array

data_array = np.vstack(data_list)#converting to numpy array, (#total trails , #no of channels, #length of signal) 

In [6]:
features=[] #stores the extracted features list, features are extracted channel wise and concatenated together

for data in data_array:
    features.append(concatenate_features(data)) #calculates all features for each trial
    
features_array = np.array(features)
features_array.shape # (no of epochs/trials, #number of channels*number of features) #pehley channel key 13 uskey bad doesrey channel key 13 and so on

(9605, 247)

In [7]:
##Label
#We have features array till now time for target/label array, whats tricky about splitting the data

#i guess label array shouldnt be split for training and testing since the model would have already seen the data from the same patient  as it
# would be splitting based on epochs and epochs can be from the same patient

#creating lables 0 for healthy and 1 for pd
healthy_epochs_array_lable= [[0]*len(i) for i in healthy_epochs_array ]# create arrays with 0's, where number of zeros in each entry is equal to the number of trials in that file, for eg first entry will have 47 0's
patient_epochs_array_lable= [[1]*len(i) for i in patient_epochs_array ] # 1 instead of 0

lable_list= healthy_epochs_array_lable+ patient_epochs_array_lable

lable_array= np.hstack(lable_list) #stacking each entry 

print(len(lable_list),len(lable_array),lable_array.shape)

28 9605 (9605,)


In [8]:
##groups 
group_list=[[i]*len(j) for i,j in enumerate(data_list)]
group_array=np.hstack(group_list) #stacking 
print(len(group_list), len(group_array))

28 9605


In [9]:
##ML Libraries 

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

X=features_array
y=lable_array
groups=group_array

In [ ]:
#HyperParameter tuning of Random foresnt with Group k fold applied 

model=RandomForestClassifier()
pipe=Pipeline([('scaler',StandardScaler()),('clf',model)])
param_grid = {'clf__n_estimators': [250,300,400], 'clf__max_features': [1,2,3,4]} ##best is estimators 60

gscvRF =  GridSearchCV(pipe,param_grid, cv=4, return_train_score=False, n_jobs=-1)             
gscvRF.fit(X,y,groups=groups)

print(gscvRF.best_params_ , gscvRF.best_score_)

In [ ]:
## Exporting the model
import pickle
with open('model_RF_PD','wb') as f:
    pickle.dump(gscvRF,f) 

In [ ]:
%%capture
TSub1_epochs_list = read_data('all_SP_edf/h01.edf')
TSub1_epochs_array=np.array(TSub1_epochs_array)
TSub1_features_array= concatenate_features(TSub1_epochs_array)

model.predict(TSub1_features_array)

In [ ]:
#hyper parameter tuning for LR 
#Grid Search does permutation of all parameters (for automatic parameter tuning on scaled model ) #this also perfors iterative kfold cross validation on its own
#multi_class='auto',

model = LogisticRegression(multi_class='auto')
pipe=Pipeline([('scaler',StandardScaler()),('classifier',model)])
param_grid = [    
    {'classifier__penalty' : ['l2'],
    'classifier__C' : [300,350,400 ],
    'classifier__solver' : ['lbfgs','newton-cg','liblinear'],
    'classifier__max_iter' : [100,200]
    }
]

gscvLr=GridSearchCV(pipe,param_grid,cv=4, n_jobs=-1) # 1st parameter model(in our case scaled model), second diffrent parameters we wana tweak of model                                                       #third number of splits in k fold             
gscvLr.fit(X,y,groups=groups)

print(gscvLr.best_params_ , gscvLr.best_score_)

In [ ]:
import pickle
with open('model_Lr_PD','wb') as f:
    pickle.dump(gscvLr,f) 

In [ ]:
!pip install sklearn-rvm

In [14]:
from sklearn_rvm import EMRVR
model=EMRVR()
param_grid={}
gscvRVC=GridSearchCV(model,param_grid,cv=4, n_jobs=-1)
gscvRVC.fit(X,y,groups=groups)

C:\Users\786me\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [nan]
  warnings.warn(
C:\Users\786me\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn_rvm\em_rvm.py:267: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "
C:\Users\786me\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn_rvm\em_rvm.py:324: UserWarning: Hessian not positive definite
  warnings.warn("Hessian not positive definite")


KeyboardInterrupt: 